**Experiment 2**

This notebook contains experiments conducted in order to find out wich word2vec parameters configuration yields the best performance


In [3]:
%load_ext autoreload
%autoreload 2

import word2vec
import gensim
# import pm_parser
import sys
sys.path.append("../../")
import pm_parser
import evaluation

def preprocess_text(dataset="Training", th1=100, th2=50):
    !sed -e "s/’/'/g" -e "s/′/'/g" -e "s/''/ /g" < {dataset} | tr -c "A-Za-z'_ \n" " " > Training-norm0
    print('Finished preprocessing 1')
    word2vec.word2phrase('Training-norm0', 'Training-norm0-phrase0', threshold=th1, verbose=True)
    print('Finished word2phrase 1')
    word2vec.word2phrase('Training-norm0-phrase0', 'Training-norm0-phrase1', threshold=th2, verbose=True)
    print('Finished word2phrase 2')
    !tr A-Z a-z < Training-norm0-phrase1 > Training-norm1-phrase1
    print('Finished preprocessing 2')

def train_model(model=".model", cbow=1, size=200, window=10,sample="1e-5", min_count=5, negative=25, threads=20, iter_=15):
    word2vec.word2vec('Training-norm1-phrase1', model, cbow=cbow, min_count=min_count, size=size, window=window,sample=sample, negative=negative, threads=threads, iter_=iter_, verbose=True)
    print('Finished model training')
    
def train_in_grid(cbows=[0,1], sizes=[150,200,250], windows=[5,10,15], samples=["1e-4","1e-5","1e-6"], min_counts=[0,1,2,3]):
    for cbow in cbows:
        for size in sizes:
            for window in windows:
                for sample in samples:
                    for min_count in min_counts:
                        name = "{}_{}_{}_{}_{}.model".format(cbow,size,window,sample,min_count)
                        print('training w2v:', name)
                        train_model(model=name,cbow=cbow,size=size,window=window,sample=sample,min_count=min_count)
                        print('training finished')
    !rm Training-norm0 Training-norm0-phrase0 Training-norm0-phrase1 Training-norm1-phrase1
    print('Removed reduntant files')
    
# def evaluate_model(model, testset='../../../Shibo/word2vec/examples/Autism_Pure_Testing.txt', mwe_tokens):
# #     filename = 'Autism_Pure_Testing.txt'

#     with open(filename, 'r') as file_to_read:
#         while True:
#             text_to_annotate = file_to_read.readline()
# #             pm_parser.evaluate(text_to_annotate, phe)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First preprocess the dataset with word2phrase

In [4]:
preprocess_text(dataset='../../../Shibo/dataset/', th1=100, th2=50)

Finished preprocessing 1
Finished word2phrase 1
Finished word2phrase 2
/usr/bin/sh: Training-norm0-phrase1: No such file or directory
Finished preprocessing 2


Then, train the models using a grid of parameters

In [2]:
train_in_grid(cbows=[0,1], sizes=[150,200], windows=[15,20], samples=["1e-5"], min_counts=[0,1])

training w2v: 0_150_15_1e-5_0.model
Starting training using file Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 13.26k  Finished model training
training finished
training w2v: 0_150_15_1e-5_1.model
Starting training using file Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 13.17k  Finished model training
training finished
training w2v: 0_150_20_1e-5_0.model
Starting training using file Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 10.48k  Finished model training
training finished
training w2v: 0_150_20_1e-5_1.model
Starting training using file Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 10.96k  Finished model training
training finished
training w2v: 0_200_15_1e-5_0.model
Starting

In [15]:
import os
import shutil
def _get_files_in_folder(input_folder, ext=".xml.gz", start_file=None, end_file=None):
        currentDir = os.getcwd() if input_folder == '/' else input_folder
        start = os.path.join(currentDir, start_file) if start_file is not None else None
        end = os.path.join(currentDir, end_file) if end_file is not None else None
        files = []
        for file in os.listdir(currentDir):
            if file.endswith(ext):
                files.append(os.path.join(currentDir, file))        
        files.sort(key=lambda f: int(''.join(filter(str.isdigit, f) or -1)))
        files = files[files.index(start) if start is not None else None : files.index(end) if end is not None else None ]
        return files
    
files = _get_files_in_folder('/afs/inf.ed.ac.uk/group/project/biomednlp/msc2019/maciej/experiments/2_w2v_params/old_models', ext=".model")
cbows=[0,1] 
sizes=[150,200]
windows=[5,10]
samples=["1e-5"]
min_counts=[0,1,2]
file2move = []
for file in files:
    cbow, size, window, sample, min_count = file.split('/')[-1].split('.')[0].split('_')
#     print(cbow, size, window, sample, min_count)
    if int(cbow) in cbows and int(size) in sizes and int(window) in windows and sample in samples and int(min_count) in min_counts:
        file2move.append(file)
print(len(file2move))
for file in file2move:
    shutil.copy2(file, '/afs/inf.ed.ac.uk/group/project/biomednlp/msc2019/maciej/experiments/2_w2v_params') # complete target filename given
# shutil.copy2('/src/file.ext', '/dst/dir')

24


Finally, evaluate the trained models

In [5]:
evaluation.evaluate_folder('/', 'experiment2_gold_results.csv', ext='.model',annotated_testset_file="../../annotated_GOLD_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/16 [00:00<?, ?it/s]

Number of models to evaluate: 16


/afs/inf.ed.ac.uk/user/s12/s1248922/miniconda3/envs/mlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

 26%|██▌       | 7/27 [00:12<00:37,  1.85s/it]


100%|██████████| 16/16 [20:04<00:00, 88.75s/it]

****finished****


In [6]:
evaluation.evaluate_folder('/', 'experiment2_ncbo_results.csv', ext='.model',annotated_testset_file="../../annotated_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/16 [00:00<?, ?it/s]

Number of models to evaluate: 16



 48%|████▊     | 408/853 [01:03<01:01,  7.23it/s]


 99%|█████████▉| 848/853 [02:03<00:00,  7.67it/s]


 48%|████▊     | 407/853 [00:58<01:12,  6.19it/s]


...

 98%|█████████▊| 834/853 [02:33<00:03,  4.97it/s]


 41%|████      | 347/853 [01:08<01:19,  6.39it/s]


 84%|████████▍ | 715/853 [02:34<00:27,  4.95it/s]


100%|██████████| 16/16 [37:13<00:00, 151.89s/it]]

****finished****
